In [1]:
!python3.10 -m pip install guardrails-ai
!python3.10 -m pip install urllib3
!guardrails hub install hub://guardrails/detect_pii
!guardrails hub install hub://guardrails/teste --quiet
!python3.10 -m spacy download pt_core_news_sm
!guardrails hub install hub://guardrails/pii --quiet
!guardrails hub install hub://guardrails/nsfw_text
!guardrails hub install hub://guardrails/similar_to_document --quiet

/usr/bin/python3.10: No module named pip
/usr/bin/python3.10: No module named pip
/bin/bash: line 1: guardrails: command not found
/bin/bash: line 1: guardrails: command not found
/usr/bin/python3.10: No module named spacy
/bin/bash: line 1: guardrails: command not found
/bin/bash: line 1: guardrails: command not found
/bin/bash: line 1: guardrails: command not found


In [3]:
from guardrails.validators import (register_validator, Validator, FailResult, ValidationResult, PassResult)
from guardrails import Guard, OnFailAction
from guardrails.hub import DetectPII
from typing import Any, Dict, Optional, Callable, List

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, RecognizerResult, Pattern
import re

from transformers import BertTokenizer, BertModel
import torch
import os
import numpy as np
import datetime as dt

import unicodedata
import spacy
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

ModuleNotFoundError: No module named 'guardrails'

# Similaridade entre textos

- Aqui encontra-se métodos de similaridade entre para verificar se o modelo está ou não alucinando.

In [ ]:
# Usando BERT para usar seus Embeddings

if os.path.isdir("./bert_tokenizer") and os.path.isdir("./bert_model"):
    tokenizer = BertTokenizer.from_pretrained("./bert_tokenizer")
    model = BertModel.from_pretrained("./bert_model")
else:
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    tokenizer.save_pretrained("./bert_tokenizer")
    model.save_pretrained("./bert_model")

In [ ]:
"""
    Descrição: Remove pontuação do texto

    Args:
        text: O texto a ser tratado

    Returns:
        Retorna o texto sem símbolos de pontuação
"""
def remove_pointing(text: str) -> str:
    ans = ""
    for c in text:
          if unicodedata.category(c) != 'Po':
               ans+=c
    return ans


"""
    Descrição: Remove os acentos das palavras do texto

    Args:
        text: O texto a ser tratado

    Returns:
        Retorna o texto sem acentos
"""
def remove_accent(text: str) -> str:
    nfkd = unicodedata.normalize('NFKD', text)
    ans = ""
    for c in nfkd:
         if not unicodedata.combining(c):
              ans += c
    return ans


"""
    Descrição: Aplica lematização no texto

    Args:
        text: O texto a ser tratado

    Returns:
        Retorna o texto lematizado
"""
def lemmatize(text: str) -> str:
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])


"""
    Descrição: Remove as palavras irrelevantes do texto

    Args:
        words:  texto a ser tratado

    Returns:
        Lista de string sem as palavras irrelevantes
"""
def remove_stopwords(text: str) -> str:
    text_without_pointing_and_accent = remove_accent(remove_pointing(text))
    lemmatized_text = lemmatize(text_without_pointing_and_accent)
    words_splitted = lemmatized_text.split()
    stop_words = set(stopwords.words("portuguese"))
    relevant_text = ""
    for word in words_splitted:
        if word.lower() not in stop_words and len(word) > 2 and len(word) < 15:
            relevant_text += re.sub(r'[^a-zA-Z0-9]', '', word.lower()) + " "

    return relevant_text


"""
    Descrição:
        Retorna os embeddings do texto passado como parâmetro da função.

    Args:
        text: texto usado para criar os embeddings

    returns:
        Retorna os Embeddings.
"""
def generate_embeddings(text: str) -> List[List]:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state
    sentence_embedding = embeddings.mean(dim=1)

    return sentence_embedding


"""
    Descrição:
        Calcula a similaridae cosseno entre dois trechos de textos.
        Gera-se os embeddings para cada texto e em seguida, fazemos a similarida usando os embeddings.

    args:
        text1: texto usado para calcular a similaridade.
        text1: texto usado para calcular a similaridade.

    returns: Grau de similaridade em um intervalo fechado entre 0 (baixo) e 1 (alto).
"""
def cosine_similarity(text1: str, text2: str) -> float:
    emb_text1 = generate_embeddings(text1)
    emb_text2 = generate_embeddings(text2)

    emb_text1 = emb_text1.cpu().numpy()
    emb_text2 = emb_text2.cpu().numpy()

    dot_product = np.dot(emb_text1, emb_text2.T)
    text1Norm = np.linalg.norm(emb_text1)
    text2Norm = np.linalg.norm(emb_text2)

    return dot_product / (text1Norm * text2Norm)


"""
    Descrição:
        Divide o texto em alguns pedaços de frases.

    Args:
        text: texto a ser dividido.
        max_length: tamanho máximo de cada divisão.

    returns: Lista de frases.
"""
def split_text(text, max_length: int = 10) -> List:

    tokens = tokenizer.tokenize(text)
    chunks = [tokenizer.convert_tokens_to_string(tokens[i:i + max_length]) for i in range(0, len(tokens), max_length)]

    return chunks


"""
    Descrição:
        Calcula a similaridade entre dois textos.

    Args:
        text1: texto usado para calcular a similaridade.
        text2: texto usado para calcular a similaridade.

    returns:
        Similaridade cosseno.

"""
def similarity_between_texts(text1, text2) -> float:

    text1 = remove_stopwords(text1)
    text2 = remove_stopwords(text2)

    print(text1)
    print(text2)

    split_text1 = split_text(text1)
    split_text2 = split_text(text2)

    similarities = []

    if len(split_text1) == 0 or len(split_text2) == 0:
        return 0

    for text1 in split_text1:
        for text2 in split_text2:
            similarities.append(cosine_similarity(text1, text2)[0][0])

    media = np.mean(similarities)
    print(media, similarities)
    return media

In [ ]:
# Testes

response_eureca = """
{
    'sexo': {
        'feminino': {
            'quantidade': 194,
            'estado_civil': {
                'Casado': 2,
                'Solteiro': 186,
                'Divorciado': 1,
                '-': 5
            },
            'nacionalidades': {
                'brasileira': 194,
                'estrangeira': 0
            },
            'estados': {
                'PE': 17,
                'PB': 146,
                'RJ': 4,
                'RN': 4,
                'CE': 3,
                'SP': 7,
                'MG': 1,
                'MA': 2,
                'AL': 1,
                'PA': 1,
                'BA': 1,
                'PI': 2,
                None: 5
            },
            'idade': {
                'idade_minima': 18,
                'idade_maxima': 39,
                'media_idades': 22.47
            },
            'politica_afirmativa': {
                'L1': 14,
                'L6': 17,
                '-': 59,
                'L2': 26,
                'L5': 13,
                'L13': 1,
                'L14': 1,
                'Bon. estadual': 41,
                'L9': 1,
                'LB_PPI': 10,
                'LI_PPI': 6,
                'LB_EP': 2,
                'LI_PCD': 3
            },
            'cor': {
                'Branca': 98,
                'Parda': 87,
                'Preta': 4,
                'Não declarada': 5
            },
            'renda_per_capita_ate': {
                'renda_minima': 0.5,
                'renda_maxima': 99.0,
                'renda_media': 7.62},
                'tipo_de_ensino_medio': {
                    'Somente escola pública': 97,
                    'Somente escola privada': 96,
                    'Pública e privada, tendo ficado mais tempo em escola privada': 1
                }
            },

        'masculino': {
            'quantidade': 709,
            'estado_civil': {
                'Solteiro': 685,
                'Casado': 6,
                '-': 17,
                'Divorciado': 1
            },
            'nacionalidades': {
                'brasileira': 708,
                'estrangeira': 1
            },
            'estados': {
                'PB': 528,
                None: 21,
                'SP': 24,
                'PE': 49,
                'RJ': 22,
                'CE': 15,
                'RN': 13,
                'DF': 2,
                'TO': 1,
                'AL': 3,
                'BA': 14,
                'MA': 4,
                'PR': 1,
                'MS': 1,
                'PA': 2,
                'MG': 3,
                'PI': 4,
                'AP': 1,
                'RO': 1
            },
            'idade': {
                'idade_minima': 17,
                'idade_maxima': 44,
                'media_idades': 22.79
            },
            'politica_afirmativa': {
                'L2': 78,
                '-': 250,
                'L1': 51,
                'L6': 68,
                'L5': 56,
                'L10': 3,
                'L13': 3,
                'L9': 3,
                'Bon. estadual': 129,
                'L14': 1,
                'LI_PPI': 22,
                'LB_PPI': 22,
                'LB_PCD': 5,
                'LI_EP': 9,
                'LB_EP': 7,
                'LI_PCD': 2
            },
            'cor': {
                'Parda': 291,
                'Branca': 360,
                'Preta': 33,
                'Amarela': 7,
                'Indígena': 1,
                'Não declarada': 17
            },
            'renda_per_capita_ate': {
                'renda_minima': 0.5,
                'renda_maxima': 99.0,
                'renda_media': 6.79
            },
            'tipo_de_ensino_medio': {
                'Somente escola pública': 347,
                'Somente escola privada': 359,
                'Pública e privada, tendo ficado mais tempo em escola pública': 1,
                'Pública e privada, tendo ficado mais tempo em escola privada': 2
            }
        }
    }
}
"""

response_llm = """
        ### Alunas (Feminino)

        - **Quantidade Total**: 183
        - **Estado Civil**
        - Solteiro: 175
        - Casado:3
        - **Nacionalidade**: 100% Brasileira
        - **Principais Estados**
        -PB:139
        - PE:17
        - **Idade**:
        - Média: 21.68 anos
        - Mínima: 17 anos
        - Máxima: 39 anos
        - **Política Afirmativa**:
        - Bon. estadual: 40
        - L2: 26
        - **Cor/Raça**:
          -Branca: 92
          -Parda: 83
        - **Renda Per Capita**:
        - Média: 6.78
        - **Tipo de Ensino Médio**
        - Somente escola pública: 93
        - Somente escola privada:89

        ### Alunos (Masculino)

        ⁃ **Quantidade Total**: 658
        - **Estado Civil**:
        ⁃ Solteiro: 641
        - Casado:6
        - **Nacionalidade**: Predominantemente Brasileira (657 brasileiros, 1 estrangeiro)
        - **Principais Estados**:
        -PB:498
        -PE: 47
        ⁃ **Idade**:
        ⁃ Média: 21.87 anos
        - Minima: 17 anos
        ⁃ Máxima: 43 anos
        - **Política Afirmativa**
        - Bon. estadual: 127
        - L2:71
        - **Cor/Raca**:
        ⁃ Branca: 337
        -Parda: 271
        - **Renda Per Capita**:
        - Média: 7.05
        - **Tipo de Ensino Médio**:
        ⁃ Somente escola pública: 329
        - Somente escola privada: 327

        Essas informações destacam a distribuição de gênero, origem
        geográfica, política afirmativa, e outros aspectos demograficos e
        educacionais dos alunos.
"""

# Guardrails

- Guardrails é uma estrutura Python que ajuda a construir aplicativos de IA confiáveis, executando duas funções principais:

- Guardrails executa protetores de entrada/saída em sua aplicação que detectam, quantificam e mitigam a presença de tipos específicos de riscos. Para ver o conjunto completo de riscos, confira Guardrails Hub.
- Guardrails ajudam você a gerar dados estruturados de LLMs.

In [ ]:
"""
    Descrição:
        Calcula a similaridade entre dois textos e valida se a similaridade está acima de um limiar definido pelo usuário.

    Returns:
        Retorna a similaridade entre dois textos.
"""
@register_validator(name="guardrails/teste", data_type="string")
class ValidadorDeSimilaridade(Validator):
    def __init__(self, texto1: str, texto2: str, match_type: Optional[str] = None, on_fail: Optional[Callable] = None):
        super().__init__(on_fail=on_fail, match_type=match_type)

        self.texto1 = texto1
        self.texto2 = texto2

    def validate(self, value: Any, metadata: Dict = {}) -> ValidationResult:
        similarity = float(f"{similarity_between_texts(self.texto1, self.texto2):.1f}")
        print(similarity)

        ideal_similarity = 0.7
        if similarity < ideal_similarity:
            print(f"{value}: Similaridade baixa {similarity} (menor que {ideal_similarity})")
            return FailResult(error_message="Erro")

        print(f"{value}: Similaridade alta de {similarity} (igual ou acima de {ideal_similarity})")
        return PassResult()

In [ ]:
def teste(texto1, texto2):
    guard = Guard().use(
        ValidadorDeSimilaridade(texto1=texto1, texto2=texto2)
    )

    try:
        guard.parse("Agente Inteligente").model_validate
        print("Passou no teste de similaridade cosseno!")
    except Exception as e:
        print("Ocorreu um erro: ", e)

In [ ]:
texto1 = """
lula
"""

texto2 = """
carro
"""

teste(texto1=texto1, texto2=texto2)

In [ ]:
texto1 = """
de onde sao os alunos de ciencia da computacao? liste todos os estados
"""

texto2 = """
Os alunos de Ciência da Computação vêm de diversos estados,
incluindo:


Pernambuco (PE)
Rio de Janeiro (RJ)
São Paulo (SP)
Minas Gerais (MG)
Bahia (BA)


Esses sao alguns exemplos dos estados de origem dos alunos."""

teste(texto1=texto1, texto2=texto2)

In [ ]:
texto1 = "Lula é o presidente do Brasil"
texto2 = "Lula não é o presidente do Brasil"

teste(texto1=texto1, texto2=texto2)

In [ ]:
"""
    Descrição:
        A partir de um texto passado através do método parse, este guardrail camufla a matricula do usuário no texto e retorna o texto camuflado.
"""
@register_validator(name="guardrails/enrollment", data_type="string")
class PIIValidator(Validator):
    def __init__(self, on_match: Optional[str] = None, on_fail: Optional[Callable] = None):
        super().__init__(on_match=on_match, on_fail=on_fail)

    def validate(self, value: str, metadata: Dict = {}) -> ValidationResult:
        year = str(dt.datetime.now().year)[2:]

        enrollment = r"\b\d{1}[1-" + year[0] + r"]" + r"[0-" + year[1] + r"]" + r"[12]\d{5}\b"
        enrollment_pattern = Pattern(name="Enrollment_Pattern", regex=enrollment, score=0.6)

        enrollment_recognizer = PatternRecognizer(name="MATRICULA", patterns=[enrollment_pattern], supported_entity="Enrollment_Pattern", supported_language="en")

        analyzer = AnalyzerEngine()
        analyzer.registry.add_recognizer(enrollment_recognizer)

        results = analyzer.analyze(text=value, entities=["Enrollment_Pattern"], language="en")

        if results:
            for result in results:
                start, end = result.start, result.end
                value = value[:start] + "<MATRICULA>" + value[end:]
            return FailResult(error_message=value)

        return PassResult()

In [ ]:
guardas_eureca = Guard().use(PIIValidator)

try:
    guardas_eureca.parse("A minha matricula é o seguinte: 221199999, busque informaões sobre ela").model_validate
except Exception as e:
    print(e)

In [ ]:

"""
Descrição:
    Guardrail para dado um texto, camuflar o seu CPF.
"""
@register_validator(name="guardrails/cpf", data_type="string")
class PIIValidatorCPF(Validator):
    def __init__(self, on_match: Optional[str] = None, on_fail: Optional[Callable] = None):
        super().__init__(on_match=on_match, on_fail=on_fail)


    """
    Descrição:
        Verifica se a possivel sequência de números é ou não um CPF através do calculo dos digitos verificadores dos 9 primeiros números.

    Args:
        cpf_user: CPF a ser validado.

    returns:
        Retorna um valor booleano.
        True se a sequência de números é um CPF e False, caso contrário.
    """
    def validate_digit_verificator(self, cpf_user):
        if len(set(cpf_user)) == 1: return False

        cpf = cpf_user[:9]
        sum = 0
        for i in range(1, len(cpf) + 1):
            sum += i * int(cpf[i - 1])
        first_digit_verificator = sum % 11

        if first_digit_verificator == 10:
            first_digit_verificator = 0


        sum = 0
        cpf = cpf + str(first_digit_verificator)
        for i in range(len(cpf)):
            sum += i * int(cpf[i])
        second_digit_verificator = sum % 11

        if second_digit_verificator == 10:
            second_digit_verificator = 0

        cpf = cpf[:9] + str(first_digit_verificator) + str(second_digit_verificator)
        return cpf == cpf_user


    """
    Descrição:
        Camufla o CPF de um texto, buscando as posições onde os possiveis CPFs ocorrem.

    Args:
        text: string do cpf a ser validado.

    returns:
        Retorna um texto com o CPF camuflado.
    """
    def replace_valid_cpfs(self, text) -> str:
        cpf_pattern =r"\b\d{3}[^a-zA-Z]*\d{3}[^a-zA-Z]*\d{3}[^a-zA-Z]*\d{2}\b"

        def validate_and_replace(match):
            cpf = match.group(0)
            cpf_numbers = re.sub(r"[^\d]", "", cpf)

            if self.validate_digit_verificator(cpf_numbers):
                return "<CPF>"
            return cpf

        return re.sub(cpf_pattern, validate_and_replace, text)

    def validate(self, value: str, metadata: Dict = {}) -> ValidationResult:
        result = self.replace_valid_cpfs(value)

        if "<CPF>" in result:
            return FailResult(error_message=result)
        return PassResult()


guardas_eureca = Guard().use(PIIValidatorCPF)

try:
    result = guardas_eureca.parse("O meu CPF é o seguinte: 99999999999, busque informaões sobre ele").model_validate
except Exception as e:
    print(str(e)[41:])

In [ ]:
guarda_email = Guard().use(DetectPII(pii_entities="pii", on_fail="fix"))

text = """
Meus emails são demo@lol.com ou dominio@gmail.com ou dominio@hotmail.com e dominio@hotmail.com.br;
e os meus números de telefones são esses (99) 999999999 ou (99)999999999 ou 99999999999 e 99999999999,
busque no google.com.br ou uol.com.
"""
output = guarda_email.parse(
    llm_output=text,
    metadata={"pii_entities": ["EMAIL_ADDRESS", "URL", "PHONE_NUMBER"]},
)

print(output.validated_output)

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline

"""
    Descrição:
        Valida se a LLM alucinou baseado na saída da LLM e os dados do Eureca.
"""
@register_validator(name="hallucination_detector", data_type="string")
class HallucinationValidation(Validator):
    def __init__(
            self,
            embedding_model: Optional[str] = None,
            entailment_model: Optional[str] = None,
            sources: Optional[List[str]] = None,
            **kwargs
        ):
        if embedding_model is None:
            embedding_model = 'all-MiniLM-L6-v2'
        self.embedding_model = SentenceTransformer(embedding_model)

        self.sources = sources

        if entailment_model is None:
            entailment_model = 'GuardrailsAI/finetuned_nli_provenance'
        self.nli_pipeline = pipeline("text-classification", model=entailment_model)

        super().__init__(**kwargs)

    def validate(
        self, value: str, metadata: Optional[Dict[str, str]] = None
    ) -> ValidationResult:
        sentences = self.split_sentences(value)

        relevant_sources = self.find_relevant_sources(sentences, self.sources)

        entailed_sentences = []
        hallucinated_sentences = []
        for sentence in sentences:
            is_entailed = self.check_entailment(sentence, relevant_sources)
            if not is_entailed:
                hallucinated_sentences.append(sentence)
            else:
                entailed_sentences.append(sentence)

        if len(hallucinated_sentences) > 0:
            return FailResult(
                error_message=f"Alucinação detectada: {hallucinated_sentences}",
            )

        return PassResult()

    def split_sentences(self, text: str) -> List[str]:
        return nltk.sent_tokenize(text)

    def find_relevant_sources(self, sentences: str, sources: List[str]) -> List[str]:
        source_embeds = self.embedding_model.encode(sources)
        sentence_embeds = self.embedding_model.encode(sentences)

        relevant_sources = []

        for sentence_idx in range(len(sentences)):
            sentence_embed = sentence_embeds[sentence_idx, :].reshape(1, -1)
            cos_similarities = np.sum(np.multiply(source_embeds, sentence_embed), axis=1)
            top_sources = np.argsort(cos_similarities)[::-1][:5]
            top_sources = [i for i in top_sources if cos_similarities[i] > 0.8]

            relevant_sources.extend([sources[i] for i in top_sources])

        return relevant_sources

    def check_entailment(self, sentence: str, sources: List[str]) -> bool:
        for source in sources:
            output = self.nli_pipeline({'text': source, 'text_pair': sentence})
            if output['label'] == 'entailment':
                return True
        return False

In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = "GuardrailsAI/finetuned_nli_provenance"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

save_directory = "./saved_models/finetuned_nli_provenance"

tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

print(f"Modelo salvo em: {save_directory}")

In [ ]:
save_directory = "./saved_models/finetuned_nli_provenance"

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModel.from_pretrained(save_directory)

In [ ]:
guard = Guard().use(
    HallucinationValidation(
        embedding_model='all-MiniLM-L6-v2',
        entailment_model='./saved_models/finetuned_nli_provenance',
        sources=[response_eureca],
        on_fail=OnFailAction.EXCEPTION
    )
)

guard.validate(response_llm)

In [ ]:
from transformers import pipeline

CLASSIFIER = pipeline(
    "zero-shot-classification",
    model='facebook/bart-large-mnli',
    hypothesis_template="Verifique em qual tópico o texto acima melhor se adequa: {}.",
    multi_label=True,
)

dataset = [
    ("Quais são as disciplinas oferecidas no curso de Matemática", "disciplina"),
    ("Quantos cursos a universidade oferece", "curso"),
    ("Qual o calendário acadêmico deste semestre", "período"),
    ("Onde os alunos do curso de Engenharia são matriculados", "estudante"),
    ("Quantos professores tem em toda a universidade", "professor"),
    ("Quantos professores lecionam no curso de Psicologia", "professor"),
    ("Qual livro aborda o estudo de inteligência artificial", "livro"),
    ("Quais são os livros recomendados para o curso de Direito", "livro"),
    ("Quais as disciplinas do curso de Engenharia Civil", "disciplina"),
    ("Qual o período de férias da universidade", "período"),
    ("Quantos alunos estão matriculados no curso de ciencia da computacao", "estudante"),
    ("Quais são os livros que falam sobre estatística", "livro"),
]

class TopicTest:
    def __init__(self, dataset):
        self.dataset = dataset
        self.vocab_size = len(dataset)
        self.result = None
        self.topics = []
        self.test()

    def test(self):
        total_success = 0
        total_errors = 0
        errors = []
        self.topics = list(set([topic for _, topic in self.dataset]))

        for (input, output) in dataset:
            classified_output = CLASSIFIER(input, self.topics)
            classifier = classified_output['labels'][0]

            if classifier == output:
                total_success += 1
            else:
                errors.append({"sentence": input, "output_expected": output, "output_model": classifier, "score": classified_output['scores'][0]})
                total_errors += 1

        success_rate = total_success / self.vocab_size * 100
        errors_rate = total_errors / self.vocab_size * 100

        self.result = {
            "total_success": total_success,
            "total_errors": total_errors,
            "success_rate": success_rate,
            "errors_rate": errors_rate,
            "errors_ocurred": errors
        }

    def explain_test(self):
        print(f'Obteve {self.result["total_success"]} acertos ({self.result["success_rate"]:.2f}%) e {self.result["total_errors"]} erro(s) ({self.result["errors_rate"]:.2f}%) de {tester.vocab_size} amostras')

tester = TopicTest(dataset=dataset)
print(tester.result)
tester.explain_test()

input = response_llm
classified_output = CLASSIFIER(input, ["disciplina", "curso", "período", "estudante", "professor", "livro"])

In [ ]:
from guardrails.hub import NSFWText
from guardrails import Guard

guard = Guard().use(NSFWText, threshold=0.8, validation_method="sentence", on_fail="exception")

guard.validate(
    "Christopher Nolan's Tenet is a mind-bending action thriller that will keep you on the edge of your seat. The film is a must-watch for all Nolan fans."
)

try:
    guard.validate(
        ""
    )
except Exception as e:
    print(e)

In [ ]:
# Import Guard and Validator
from guardrails import Guard
from guardrails.hub import SimilarToDocument

# Initialize The Guard with this validator
guard = Guard().use(
    SimilarToDocument,
    document="""
    Large language models (LLM) are very large deep learning models that are pre-trained on vast amounts of data.
    The underlying transformer is a set of neural networks that consist of an encoder and a decoder with self-attention capabilities.
    The encoder and decoder extract meanings from a sequence of text and understand the relationships between words and phrases in it.
    Transformer LLMs are capable of unsupervised training, although a more precise explanation is that transformers perform self-learning.
    It is through this process that transformers learn to understand basic grammar, languages, and knowledge.
    """,
    threshold=0.7,
    model="all-MiniLM-L6-v2",
    on_fail="exception",
)

# Test passing response
guard.validate(
    """
    Large Language Models (LLMs) are a type of neural network that can be trained on large amounts of text
    data to generate human-like text. These models have been used in a variety of applications, including
    machine translation, text summarization, and question answering.
    """
)  # Pass

try:
    # Test failing response
    guard.validate(
        """
        Graph neural networks (GNNs) are specialized neural networks that can operate on graph data
        structures. These networks are designed to capture the relationships between nodes in a graph
        and can be used for a variety of tasks, including node classification, link prediction, and graph classification.
        """
    )  # Fail
except Exception as e:
    print(e)

In [ ]:
eureca_response = """
{
    'sexo': {
        'feminino': {
            'quantidade': 194,
            'estado_civil': {
                'Casado': 2,
                'Solteiro': 186,
                'Divorciado': 1,
                '-': 5
            },
            'nacionalidades': {
                'brasileira': 194,
                'estrangeira': 0
            },
            'estados': {
                'PE': 17,
                'PB': 146,
                'RJ': 4,
                'RN': 4,
                'CE': 3,
                'SP': 7,
                'MG': 1,
                'MA': 2,
                'AL': 1,
                'PA': 1,
                'BA': 1,
                'PI': 2,
                None: 5
            },
            'idade': {
                'idade_minima': 18,
                'idade_maxima': 39,
                'media_idades': 22.47
            },
            'politica_afirmativa': {
                'L1': 14,
                'L6': 17,
                '-': 59,
                'L2': 26,
                'L5': 13,
                'L13': 1,
                'L14': 1,
                'Bon. estadual': 41,
                'L9': 1,
                'LB_PPI': 10,
                'LI_PPI': 6,
                'LB_EP': 2,
                'LI_PCD': 3
            },
            'cor': {
                'Branca': 98,
                'Parda': 87,
                'Preta': 4,
                'Não declarada': 5
            },
            'renda_per_capita_ate': {
                'renda_minima': 0.5,
                'renda_maxima': 99.0,
                'renda_media': 7.62},
                'tipo_de_ensino_medio': {
                    'Somente escola pública': 97,
                    'Somente escola privada': 96,
                    'Pública e privada, tendo ficado mais tempo em escola privada': 1
                }
            },

        'masculino': {
            'quantidade': 709,
            'estado_civil': {
                'Solteiro': 685,
                'Casado': 6,
                '-': 17,
                'Divorciado': 1
            },
            'nacionalidades': {
                'brasileira': 708,
                'estrangeira': 1
            },
            'estados': {
                'PB': 528,
                None: 21,
                'SP': 24,
                'PE': 49,
                'RJ': 22,
                'CE': 15,
                'RN': 13,
                'DF': 2,
                'TO': 1,
                'AL': 3,
                'BA': 14,
                'MA': 4,
                'PR': 1,
                'MS': 1,
                'PA': 2,
                'MG': 3,
                'PI': 4,
                'AP': 1,
                'RO': 1
            },
            'idade': {
                'idade_minima': 17,
                'idade_maxima': 44,
                'media_idades': 22.79
            },
            'politica_afirmativa': {
                'L2': 78,
                '-': 250,
                'L1': 51,
                'L6': 68,
                'L5': 56,
                'L10': 3,
                'L13': 3,
                'L9': 3,
                'Bon. estadual': 129,
                'L14': 1,
                'LI_PPI': 22,
                'LB_PPI': 22,
                'LB_PCD': 5,
                'LI_EP': 9,
                'LB_EP': 7,
                'LI_PCD': 2
            },
            'cor': {
                'Parda': 291,
                'Branca': 360,
                'Preta': 33,
                'Amarela': 7,
                'Indígena': 1,
                'Não declarada': 17
            },
            'renda_per_capita_ate': {
                'renda_minima': 0.5,
                'renda_maxima': 99.0,
                'renda_media': 6.79
            },
            'tipo_de_ensino_medio': {
                'Somente escola pública': 347,
                'Somente escola privada': 359,
                'Pública e privada, tendo ficado mais tempo em escola pública': 1,
                'Pública e privada, tendo ficado mais tempo em escola privada': 2
            }
        }
    }
}
"""

llm_response = """
        ### Alunas (Feminino)

        - **Quantidade Total**: 183
        - **Estado Civil**
        - Solteiro: 175
        - Casado:3
        - **Nacionalidade**: 100% Brasileira
        - **Principais Estados**
        -PB:139
        - PE:17
        - **Idade**:
        - Média: 21.68 anos
        - Mínima: 17 anos
        - Máxima: 39 anos
        - **Política Afirmativa**:
        - Bon. estadual: 40
        - L2: 26
        - **Cor/Raça**:
          -Branca: 92
          -Parda: 83
        - **Renda Per Capita**:
        - Média: 6.78
        - **Tipo de Ensino Médio**
        - Somente escola pública: 93
        - Somente escola privada:89

        ### Alunos (Masculino)

        ⁃ **Quantidade Total**: 658
        - **Estado Civil**:
        ⁃ Solteiro: 641
        - Casado:6
        - **Nacionalidade**: Predominantemente Brasileira (657 brasileiros, 1 estrangeiro)
        - **Principais Estados**:
        -PB:498
        -PE: 47
        ⁃ **Idade**:
        ⁃ Média: 21.87 anos
        - Minima: 17 anos
        ⁃ Máxima: 43 anos
        - **Política Afirmativa**
        - Bon. estadual: 127
        - L2:71
        - **Cor/Raca**:
        ⁃ Branca: 337
        -Parda: 271
        - **Renda Per Capita**:
        - Média: 7.05
        - **Tipo de Ensino Médio**:
        ⁃ Somente escola pública: 329
        - Somente escola privada: 327

        Essas informações destacam a distribuição de gênero, origem
        geográfica, política afirmativa, e outros aspectos demograficos e
        educacionais dos alunos.
"""



In [ ]:
from guardrails import Guard
from guardrails.hub import SimilarToDocument

guard = Guard().use(
    SimilarToDocument,
    document=eureca_response,
    threshold=0.7,
    model="all-MiniLM-L6-v2",
    on_fail="exception",
)

guard.validate(llm_response)

try:
    guard.validate(
        """
        Graph neural networks (GNNs) are specialized neural networks that can operate on graph data
        structures. These networks are designed to capture the relationships between nodes in a graph
        and can be used for a variety of tasks, including node classification, link prediction, and graph classification.
        """
    )
except Exception as e:
    print(e)